# Content-based Book Recommender System

In [1]:
%pip install rake_nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/reddragon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/reddragon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import io
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

/var/folders/b6/wvsnhxp56z97rld_stlsjcrh0000gn/T/ipykernel_4038/1376994740.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
# read datasets
# books_df = pd.read_csv('/Users/reddragon/Documents/SMU/Y2S2/IS215 Digital Business/project /book_recommendation_dataset/Books.csv', nrows=10000)
# users_df = pd.read_csv('/Users/reddragon/Documents/SMU/Y2S2/IS215 Digital Business/project /book_recommendation_dataset/Users.csv')
# ratings_df = pd.read_csv('/Users/reddragon/Documents/SMU/Y2S2/IS215 Digital Business/project /book_recommendation_dataset/Ratings.csv')

In [5]:
# read dataset
bestselling_df = pd.read_csv('/Users/reddragon/Documents/SMU/Y2S2/IS215 Digital Business/project /best-selling-books.csv')
bestselling_df.tail()

,Book,Author(s),Original language,First published,Approximate sales in millions,Genre
169,The Goal,Eliyahu M. Goldratt,English,1984,10.0,NaN
170,Fahrenheit 451,Ray Bradbury,English,1953,10.0,NaN
171,Angela's Ashes,Frank McCourt,English,1996,10.0,NaN
172,The Story of My Experiments with Truth (સત્યના...,Mohandas Karamchand Gandhi,Gujarati,1929,10.0,NaN
173,Bridget Jones's Diary,Helen Fielding,English,1996,10.0,NaN


In [6]:
# user_ratings_df = ratings_df.merge(users_df, left_on = 'User-ID', right_on = 'User-ID')
# user_ratings_df.head()

In [7]:
# book_user_rating = books_df.merge(user_ratings_df, left_on = 'ISBN',right_on = 'ISBN')
# book_user_rating.reset_index(drop=True, inplace = True)
# book_user_rating.head()

In [107]:
books_df = bestselling_df[['Book', 'Author(s)', 'Original language', 'Genre']]
books_df.dropna(inplace=True)

/var/folders/b6/wvsnhxp56z97rld_stlsjcrh0000gn/T/ipykernel_4038/286874660.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_df.dropna(inplace=True)


In [108]:
books2_df = books_df
books2_df.head()

,Book,Author(s),Original language,Genre
0,A Tale of Two Cities,Charles Dickens,English,Historical fiction
1,The Little Prince (Le Petit Prince),Antoine de Saint-Exupéry,French,Novella
2,Harry Potter and the Philosopher's Stone,J. K. Rowling,English,Fantasy
3,And Then There Were None,Agatha Christie,English,Mystery
4,Dream of the Red Chamber (紅樓夢),Cao Xueqin,Chinese,Family saga


<h4> Transforming Variables </h4>

<h5> for Authors </h5>

In [109]:
# Split the author names by commas and lowercase each word
books2_df['Author(s)'] = books2_df['Author(s)'].apply(lambda x: [name.lower() for name in x.split(', ')])

# Join the lowercase words into a single string for each author and update the 'Author(s)' column
books2_df['Author(s)'] = books2_df['Author(s)'].apply(lambda x: ''.join(x))

# Remove the spaces between each word in the string
books2_df['Author(s)'] = books2_df['Author(s)'].apply(lambda x: ''.join(x.split()))

# Display the updated 'Author(s)' column
books2_df['Author(s)'].head()

/var/folders/b6/wvsnhxp56z97rld_stlsjcrh0000gn/T/ipykernel_4038/404380057.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books2_df['Author(s)'] = books2_df['Author(s)'].apply(lambda x: [name.lower() for name in x.split(', ')])
/var/folders/b6/wvsnhxp56z97rld_stlsjcrh0000gn/T/ipykernel_4038/404380057.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books2_df['Author(s)'] = books2_df['Author(s)'].apply(lambda x: ''.join(x))
/var/folders/b6/wvsnhxp56z97rld_stlsjcrh0000gn/T/ipykernel_4038/404380057.py

0            charlesdickens
1    antoinedesaint-exupéry
2               j.k.rowling
3            agathachristie
4                 caoxueqin
Name: Author(s), dtype: object

<h5> for Language </h5>

In [110]:
# Split the author names by commas and lowercase each word
books2_df['Original language'] = books2_df['Original language'].apply(lambda x: [name.lower() for name in x.split(', ')])

# Join the lowercase words into a single string for each author and update the 'Author(s)' column
books2_df['Original language'] = books2_df['Original language'].apply(lambda x: ''.join(x))

# Display the updated 'Author(s)' column
books2_df['Original language'].head()

/var/folders/b6/wvsnhxp56z97rld_stlsjcrh0000gn/T/ipykernel_4038/311120575.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books2_df['Original language'] = books2_df['Original language'].apply(lambda x: [name.lower() for name in x.split(', ')])
/var/folders/b6/wvsnhxp56z97rld_stlsjcrh0000gn/T/ipykernel_4038/311120575.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books2_df['Original language'] = books2_df['Original language'].apply(lambda x: ''.join(x))


0    english
1     french
2    english
3    english
4    chinese
Name: Original language, dtype: object

<h5> for Genre </h5>

In [111]:
# Split the author names by commas and lowercase each word
books2_df['Genre'] = books2_df['Genre'].apply(lambda x: [name.lower() for name in x.split(', ')])

# Join the lowercase words into a single string for each author and update the 'Author(s)' column
books2_df['Genre'] = books2_df['Genre'].apply(lambda x: ''.join(x))

# Remove the spaces between each word in the string
books2_df['Genre'] = books2_df['Genre'].apply(lambda x: ''.join(x.split()))

# Display the updated 'Author(s)' column
books2_df['Genre'].head()

/var/folders/b6/wvsnhxp56z97rld_stlsjcrh0000gn/T/ipykernel_4038/2652369464.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books2_df['Genre'] = books2_df['Genre'].apply(lambda x: [name.lower() for name in x.split(', ')])
/var/folders/b6/wvsnhxp56z97rld_stlsjcrh0000gn/T/ipykernel_4038/2652369464.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books2_df['Genre'] = books2_df['Genre'].apply(lambda x: ''.join(x))
/var/folders/b6/wvsnhxp56z97rld_stlsjcrh0000gn/T/ipykernel_4038/2652369464.py:8: SettingWith

0    historicalfiction
1              novella
2              fantasy
3              mystery
4           familysaga
Name: Genre, dtype: object

In [112]:
books2_df.head()

,Book,Author(s),Original language,Genre
0,A Tale of Two Cities,charlesdickens,english,historicalfiction
1,The Little Prince (Le Petit Prince),antoinedesaint-exupéry,french,novella
2,Harry Potter and the Philosopher's Stone,j.k.rowling,english,fantasy
3,And Then There Were None,agathachristie,english,mystery
4,Dream of the Red Chamber (紅樓夢),caoxueqin,chinese,familysaga


<h4> Bag of Words </h4>


In [113]:
# Title should be omitted from bag of words creation
columns = books2_df.columns[1:]

books2_df['bag_of_words'] = ''

for index, row in books2_df.iterrows():
    words = ''
    for col in columns:
            words = words + str(row[col])+ ' '
    books2_df.loc[index, 'bag_of_words'] = words
# let's keep only the title and the bag of words in the dataframe
books2_df = books2_df[['Book','bag_of_words']]

/var/folders/b6/wvsnhxp56z97rld_stlsjcrh0000gn/T/ipykernel_4038/1624500921.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books2_df['bag_of_words'] = ''


In [114]:
books2_df.head()

,Book,bag_of_words
0,A Tale of Two Cities,charlesdickens english historicalfiction
1,The Little Prince (Le Petit Prince),antoinedesaint-exupéry french novella
2,Harry Potter and the Philosopher's Stone,j.k.rowling english fantasy
3,And Then There Were None,agathachristie english mystery
4,Dream of the Red Chamber (紅樓夢),caoxueqin chinese familysaga


<h4> Cosine Similarity Matrix </h4>

In [115]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(books2_df['bag_of_words'])

In [116]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list that can be used to match the indexes
indices = pd.Series(books2_df['Book'])
indices[:5]

0                        A Tale of Two Cities
1         The Little Prince (Le Petit Prince)
2    Harry Potter and the Philosopher's Stone
3                    And Then There Were None
4              Dream of the Red Chamber (紅樓夢)
Name: Book, dtype: object

In [117]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
print(cosine_sim)

[[1.         0.         0.33333333 ... 0.33333333 0.33333333 0.33333333]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.33333333 0.         1.         ... 0.33333333 0.33333333 0.33333333]
 ...
 [0.33333333 0.         0.33333333 ... 1.         0.33333333 0.33333333]
 [0.33333333 0.         0.33333333 ... 0.33333333 1.         0.33333333]
 [0.33333333 0.         0.33333333 ... 0.33333333 0.33333333 1.        ]]


<h4> Run the Model </h4>

In [118]:
# function that takes in movie title as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):

    recommended_books = []

    # getting the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)

    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_books.append(list(books2_df['Book'])[i])

    return recommended_books

In [120]:
recommendations('The Hunger Games')

['Mockingjay',
 'Catching Fire',
 'The Book Thief',
 'The Lost Symbol',
 'Love Story',
 'The Godfather',
 'The Girl on the Train',
 'The Fault in Our Stars',
 'Lust for Life',
 'A Brief History of Time']